# Best practices for working with partitions #

- The consumers of the topic need to aggregate by some attribute of the data
- The consumers need some sort of ordering guarantee
- Another resource is a bottleneck and you need to shard data
- You want to concentrate data for efficiency of storage and/or indexing

**Understand the data rate of your partitions to ensure you have the correct retention space**

**Unless you have architectural needs that require you to do otherwise, use random partitioning when writing to topics** -  It is particularly suited for stateless or “embarrassingly parallel” services.

To get an efficiency boost, the default partitioner in Kafka from version 2.4 onwards uses a “sticky” algorithm, which groups all messages to the same random partition for a batch.

**Consumer partition assignment** 

**eager protocol** By default, when a rebalance happens, all consumers drop their partitions and are reassigned new ones 

**StickyAssignor** To reduce this partition shuffling on stateful services, you can use the StickyAssignor. This assignor makes some attempt to keep partition numbers assigned to the same instance, as long as they remain in the group, while still evenly distributing the partitions across members. Because partitions are always revoked at the start of a rebalance, the consumer client code must track whether it has kept/lost/gained partitions, if partition moves are important to the logic of the application. 

**CooperativeStickyAssignor** From Kafka release 2.4 and later. Rather than always revoking all partitions at the start of a rebalance, the consumer listener only gets the difference in partitions revoked, as assigned over the course of the rebalance. The rebalances as a whole do take longer, and in our application we need to optimize for shortening the time of rebalances when a partition does move. That's why we stayed with using the “eager” protocol under the StickyPartitioner for our aggregator service.

# Best practices for working with consumers #

**Tune your consumer socket buffers for high-speed ingest**

The parameter is **receive.buffer.bytes**, which defaults to 64kB(Kafka 0.10.x) . The parameter is socket.receive.buffer.bytes defaults to 100kB (Kafka 0.8.x).

For high-bandwidth networks (10 Gbps or higher) with latencies of 1 millisecond or more, consider setting the socket buffers to 8 or 16 MB. If memory is scarce, consider 1 MB. 
You can also use a value of -1, which lets the underlying operating system tune the buffer size based on network conditions. 
However, the automatic tuning might not occur fast enough for consumers that need to start "hot."

**Design high-throughput consumers to implement back-pressure when warranted**

 it is better to consume only what you can process efficiently than it is to consume so much that your process grinds to a halt and then drops out of the consumer group.
 Consumers should consume into fixed-sized buffers (see the Disruptor pattern), preferably off-heap if running in a Java virtual machine (JVM).

 A fixed-size buffer will prevent a consumer from pulling so much data onto the heap that the JVM spends all of its time performing garbage collection instead of the work you want to achieve—which is processing messages.

**When running consumers on a JVM, be wary of the impact that garbage collection can have on your consumers.** 

For example, long garbage collection pauses can result in dropped ZooKeeper sessions or consumer-group rebalances. 
The same is true for brokers, which risk dropping out of the cluster if garbage collection pauses are too long.


# Best practices for working with producers #

**Configure your producer to wait for acknowledgments**

 This is how the producer knows that the message has actually made it to the partition on the broker. In Kafka 0.10.x, the settings is acks; in 0.8.x, it’s request.required.acks. 
 
 Kafka provides fault-tolerance via replication so the failure of a single node or a change in partition leadership does not affect availability. 
 If you configure your producers without acks (otherwise known as "fire and forget"), messages can be silently lost.

**Configure retries on your producers** (default=3)

For applications where data-loss cannot be tolerated, consider Integer.MAX_VALUE (effectively, infinity). This guards against situations where the broker leading the partition isn't able to respond to a produce request right away.

**For high-throughput producers, tune buffer sizes, particularly buffer.memory and batch.size (which is counted in bytes)** 

Because batch.size is a per-partition setting, producer performance and memory usage can be correlated with the number of partitions in the topic. The values here depend on several factors: producer data rate (both the size and number of messages), the number of partitions you are producing to, and the amount of memory you have available. Keep in mind that larger buffers are not always better because if the producer stalls for some reason (say, one leader is slower to respond with acknowledgments), having more data buffered on-heap could result in more garbage collection.

2.  Instrument your application to track metrics such as number of produced messages, average produced message size, and number of consumed messages.



# Best practices for working with brokers #

**Compacted topics require memory and CPU resources on your brokers**

 Log compaction needs both heap (memory) and CPU cycles on the brokers to complete successfully, and failed log compaction puts brokers at risk from a partition that grows unbounded. You can tune log.cleaner.dedupe.buffer.size and log.cleaner.threads on your brokers, but keep in mind that these values affect heap usage on the brokers. If a broker throws an OutOfMemoryError exception, it will shut down and potentially lose data. The buffer size and thread count will depend on both the number of topic partitions to be cleaned and the data rate and key size of the messages in those partitions. As of Kafka version 0.10.2.1, monitoring the log-cleaner log file for ERROR entries is the surest way to detect issues with log cleaner threads.

**Monitor your brokers for network throughput—both transmit (TX) and receive (RX)—as well as disk I/O, disk space, and CPU usage. Capacity planning is a key part of maintaining cluster performance**

**Distribute partition leadership among brokers in the cluster** 

 Leadership requires a lot of network I/O resources. For example, when running with replication factor 3, a leader must receive the partition data, transmit two copies to replicas, plus transmit to however many consumers want to consume that data. So, in this example, being a leader is at least four times as expensive as being a follower in terms of network I/O used. Leaders may also have to read from disk; followers only write.

**Don’t neglect to monitor your brokers for in-sync replica (ISR) shrinks, under-replicated partitions, and unpreferred leaders** 

 These are signs of potential problems in your cluster. For example, frequent ISR shrinks for a single partition can indicate that the data rate for that partition exceeds the leader’s ability to service the consumer and replica threads.

**Modify the Apache Log4j properties as needed**

 Kafka broker logging can use an excessive amount of disk space. However, don’t forgo logging completely—broker logs can be the best, and sometimes only—way to reconstruct the sequence of events after an incident.

**Either disable automatic topic creation or establish a clear policy regarding the cleanup of unused topics**

 For example, if no messages are seen for x days, consider the topic defunct and remove it from the cluster. This will avoid the creation of additional metadata within the cluster that you’ll have to manage.

**For sustained, high-throughput brokers, provision sufficient memory to avoid reading from the disk subsystem**

 Partition data should be served directly from the operating system’s file system cache whenever possible. However, this means you’ll have to ensure your consumers can keep up; a lagging consumer will force the broker to read from disk.

**For a large cluster with high-throughput service level objectives (SLOs), consider isolating topics to a subset of brokers**

 How you determine which topics to isolate will depend on the needs of your business. For example, if you have multiple online transaction processing (OLTP) systems using the same cluster, isolating the topics for each system to distinct subsets of brokers can help to limit the potential blast radius of an incident.

**Using older clients with newer topic message formats, and vice versa, places extra load on the brokers as they convert the formats on behalf of the client. Avoid this whenever possible.**

**Don’t assume that testing a broker on a local desktop machine is representative of the performance you’ll see in production** 

Testing over a loopback interface to a partition using replication factor 1 is a very different topology from most production environments. The network latency is negligible via the loopback and the time required to receive leader acknowledgements can vary greatly when there is no replication involved.